<a href="https://colab.research.google.com/github/rafaelfelixmoraes/gemini-python-study/blob/main/python/python_gemini_embedding_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exemplo/Estudo de Caso de sistema para busca em documentos usando embeddings

Inicializando bibliotecas e configs

In [44]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

from pathlib import Path
from google.colab import userdata
from IPython.display import display
from IPython.display import Markdown
import hashlib
import google.generativeai as genai
import numpy as np
import pandas as pd
import textwrap

genai.configure(api_key=userdata.get('gemini_api_key'))

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Set up the model
generation_config = {
  "temperature": 0.9,
  "top_p": 0.95,
  "top_k": 32,
  "candidate_count": 1
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = "models/embedding-001"
model_2 = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

Exemplo de embeddings

In [18]:
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

result = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(result)

{'embedding': [0.0570539, -0.03565001, -0.017294962, 0.019091211, 0.07174853, 0.0010996696, -0.040668745, -0.015815698, 0.051269732, 0.06209504, 0.021189153, 0.030685235, -0.037319556, -0.014693533, 0.004267153, -0.027338237, 0.021764165, -0.0054401406, -0.030657582, 0.0023681386, 0.0064011454, 0.0019393332, -0.045391582, -0.073757954, -0.020315424, 0.019003008, 0.02346457, -0.04051007, -0.03858568, 0.020675255, -0.058576066, 0.05430991, -0.039651725, 0.014035739, -0.02964313, -0.038898084, -0.026612258, -0.041594833, -0.003559029, -0.009682582, 0.010672286, -0.0785339, -0.00663884, 0.024225552, -0.005719359, -0.013318175, 0.040335022, 0.038090143, 0.018510686, -0.057964664, 0.022433128, 0.024026208, 0.06610874, -0.052565284, 0.0013465674, -0.0020251719, 0.027447432, -0.03595312, 0.013111223, -0.0129861655, -0.007767164, 0.026187135, -0.016783541, 0.06039292, 0.022203688, -0.05946066, -0.037069354, 5.0296014e-05, 0.01296676, 0.041830152, 0.0065639196, 0.012512145, 0.06375736, -0.058138

Lista a quantidade de dimensões no embedding

In [19]:
print(len(result['embedding']))

768


Criando os documentos que serão buscados e adicionando dentro de um DataFrame

In [20]:
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema de controle climático, use os botões e botões localizados no console central.  Temperatura: O botão de temperatura controla a temperatura dentro do carro. Gire o botão no sentido horário para aumentar a temperatura ou no sentido anti-horário para diminuir a temperatura. Fluxo de ar: O botão de fluxo de ar controla a quantidade de fluxo de ar dentro do carro. Gire o botão no sentido horário para aumentar o fluxo de ar ou no sentido anti-horário para diminuir o fluxo de ar. Velocidade do ventilador: O botão de velocidade do ventilador controla a velocidade do ventilador. Gire o botão no sentido horário para aumentar a velocidade do ventilador ou no sentido anti-horário para diminuir a velocidade do ventilador. Modo: O botão de modo permite que você selecione o modo desejado. Os modos disponíveis são: Auto: O carro ajustará automaticamente a temperatura e o fluxo de ar para manter um nível confortável. Cool (Frio): O carro soprará ar frio para dentro do carro. Heat: O carro soprará ar quente para dentro do carro. Defrost (Descongelamento): O carro soprará ar quente no para-brisa para descongelá-lo."}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e controle climático. Para usar a tela sensível ao toque, basta tocar no ícone desejado.  Por exemplo, você pode tocar no ícone \"Navigation\" (Navegação) para obter direções para o seu destino ou tocar no ícone \"Music\" (Música) para reproduzir suas músicas favoritas."}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

df = pd.DataFrame(documents)
df.columns = ["Title", "Content"]
df

,Title,Content
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


Criando os embeddings dos Documentos adicionados

In [21]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [22]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Title"], row["Content"]), axis=1)
df

,Title,Content,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.011010795, -0.026731547, -0.036728486, 0.0..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015715627, -0.040427547, 0.011117627, 0.002..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[-0.009390755, -0.022475218, -0.0024638234, 0...."


Método que realiza a consulta de um trecho de informação dentro dos documentos

In [25]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  """
    Método que cria um modelo linear para busca do embedding do documento mais próximo ao termo da consulta
  """
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  """
    Retorna o índice do token mais próximo ao contexto desejado
  """
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Content"]

In [26]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.  Park (Estacionar): Essa posição é usada quando você está estacionado. As rodas são travadas e o carro não pode se mover. Marcha à ré: Essa posição é usada para dar ré. Neutro: Essa posição é usada quando você está parado em um semáforo ou no trânsito. O carro não está em marcha e não se moverá a menos que você pressione o pedal do acelerador. Drive (Dirigir): Essa posição é usada para dirigir para frente. Low: essa posição é usada para dirigir na neve ou em outras condições escorregadias.


Integrando o uso do modelo de embedding com o modelo generativo do Gemini

In [45]:
"""
  Informando o resultado da busca feito no modelo de embedding ao modelo generativo para customização do resultado
"""
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

response = model_2.generate_content(prompt)
print("Resposta: ")
display(to_markdown(response.text))

Resposta: 


> Imagina seu Googlecar como um chefe de cozinha maneiro. Ele tem uma transmissão automática, então você só precisa dizer onde quer ir e ele vai te levar lá.
> 
> * **Estacionar:** Quando você tá relaxando no sofá, o carro fica "estacionadão". As rodinhas ficam travadinhas, tipo um cachorro prendido na coleira.
> 
> * **Marcha à ré:** Quando você precisa dar uma rézinha pra trás, tipo uma dancinha de camarão.
> 
> * **Neutro:** Se você tá numa sinaleira ou no engarrafamento, é hora do carro tirar uma soneca. Ele fica sem marcha, parado esperando você dar o "play".
> 
> * **Dirigir:** Hora de colocar o pé na estrada! Essa é a posição pra você rodar por aí como um campeão.
> 
> * **Low:** Se a pista tá escorregadia que nem uma pista de sabão, essa posição é o seu salva-vidas. O carro vai devagarzinho, tipo uma lesma com pressa.